In [1]:
import pandas as pd

In [70]:
df_basic_info = pd.read_csv("../data/processed/stocks-basic-info.csv")
df_history = pd.read_csv("../data/processed/stocks-history.csv")
df_fundaments = pd.read_csv("../data/processed/stocks-fundaments.csv")
df_right_prices = pd.read_csv("../data/processed/stocks-right-prices.csv")

In [10]:
def get_sectors():
    return df_basic_info[["SETOR", "SUBSETOR", "SEGMENTO"]].drop_duplicates()

In [16]:
def get_companies_by_segmento(segmento):
    df_tmp = df_basic_info[df_basic_info["SEGMENTO"] == segmento]
    return df_tmp[["CD_CVM", "DENOM_COMERC", "TICKERS"]]

In [83]:
def get_latest_values(ticker):
    df_history_tmp = df_history[df_history["TICKER"] == ticker]
    cd_cvm = df_history_tmp.iloc[0]["CD_CVM"]

    df_fundaments_tmp = df_fundaments[df_fundaments["CD_CVM"] == cd_cvm]
    last_dt_fim_exerc = df_fundaments_tmp["DT_FIM_EXERC"].max()

    df_fundaments_tmp = df_fundaments_tmp[df_fundaments_tmp["DT_FIM_EXERC"] == last_dt_fim_exerc]
    df_fundaments_tmp_2 = df_fundaments_tmp[df_fundaments_tmp["VL_CONTA_ROLLING_YEAR"] == -1]
    df_fundaments_tmp = df_fundaments_tmp[df_fundaments_tmp["VL_CONTA_ROLLING_YEAR"] != -1]
    df_fundaments_tmp = df_fundaments_tmp.pivot(index="DT_FIM_EXERC", columns="KPI", values="VL_CONTA_ROLLING_YEAR").reset_index()
    df_fundaments_tmp_2 = df_fundaments_tmp_2.pivot(index="DT_FIM_EXERC", columns="KPI", values="VL_CONTA").reset_index()

    df_fundaments_tmp = pd.concat([df_fundaments_tmp, df_fundaments_tmp_2], axis=1)
    df_fundaments_tmp = df_fundaments_tmp.drop("DT_FIM_EXERC", axis=1)

    df_history_tmp = df_history_tmp.tail(1).reset_index(drop=True)

    df_right_prices_tmp = df_right_prices[df_right_prices["TICKER"] == ticker]
    df_right_prices_tmp = df_right_prices_tmp.drop(["CD_CVM", "TICKER"], axis=1).reset_index(drop=True)

    return pd.concat([df_history_tmp, df_right_prices_tmp, df_fundaments_tmp], axis=1)

In [11]:
get_sectors()

,SETOR,SUBSETOR,SEGMENTO
0,Financeiro,Intermediários Financeiros,Bancos
5,Utilidade Pública,Energia Elétrica,Energia Elétrica
7,Utilidade Pública,Água e Saneamento,Água e Saneamento


In [17]:
get_companies_by_segmento(segmento="Bancos")

,CD_CVM,DENOM_COMERC,TICKERS
0,906,BANCO BRADESCO S.A.,BBDC3;BBDC4
1,22616,BANCO UBS PACTUAL S/A,BPAC3;BPAC5;BPAC11
2,922,BANCO DA AMAZONIA S.A.,BAZA3
3,1023,BANCO DO BRASIL S.A.,BBAS11;BBAS12;BBAS3
4,20532,BANCO SANTANDER (BRASIL) S.A.,SANB3;SANB4;SANB11
14,19348,ITAÚ UNIBANCO,ITUB4;ITUB3


In [84]:
get_latest_values(ticker="BBAS3")

,DATE,CD_CVM,TICKER,PRICE,PL,DIVIDEND_YIELD,DIVIDEND_PAYOUT,PVP,BAZIN,EBIT,PROFIT,CGAR_5_YEARS,EQUITY,ROE
0,2024-10-21,1023,BBAS3,26.559999,4.35535,0.094818,0.412965,0.829582,38.205378,4.108161e+10,3.494804e+10,0.150059,1.834792e+11,0.190474
